In [138]:
%load_ext autoreload

%autoreload 2

import sys
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaseEnsemble
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

sys.path.append("..")

from scripts.util import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Linear generative model

n = 200
d = 10
sigma = 0.1

X = np.random.randn(n, d)
beta = np.random.randn(d)
y = X @ beta + np.random.randn(n) * sigma

In [48]:
# Spike generative model

n = 200
d = 2
sigma = 0

X = np.random.randn(n, d)
def spike(x):
    return all(x > 0)
y = np.apply_along_axis(spike, 1, X) + np.random.randn(n) * sigma

In [ ]:
# Iris data set

iris_dataset = load_iris()

X = iris_dataset["data"]
y = iris_dataset["target"]

In [44]:
# Boston housing data set

housing_dataset = load_boston()

X = housing_dataset["data"]
y = housing_dataset["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

/Users/yanshuotan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California hous

In [141]:
decision_tree = DecisionTreeRegressor(max_depth=6)
decision_tree.fit(X_train, y_train)
tree_struct = decision_tree.tree_
stumps = make_stumps(decision_tree.tree_, normalize=True)

In [142]:
tree_transformer = TreeTransformer(decision_tree, max_components=3)
tree_transformer.fit(X_train)
transformed_feats = tree_transformer.transform(X_train)
lin_model = LinearRegression()
lin_model.fit(transformed_feats, y_train)

LinearRegression()

In [143]:
transformed_feats_test = tree_transformer.transform(X_test)
r2_score(y_test, lin_model.predict(transformed_feats_test))

0.7856420727208646

In [144]:
r2_score(y_test, decision_tree.predict(X_test))

0.808512134100913

In [145]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [146]:
rf_transformer = TreeTransformer(rf_model, max_components=5)
rf_transformer.fit(X_train)
transformed_feats = rf_transformer.transform(X_train)
lin_model_rf = LinearRegression()
lin_model_rf.fit(transformed_feats, y_train)

LinearRegression()

In [148]:
transformed_feats_test = rf_transformer.transform(X_test)
r2_score(y_test, lin_model_rf.predict(transformed_feats_test))

0.8655867799034207

In [149]:
r2_score(y_test, rf_model.predict(X_test))

0.8762961928384952

In [150]:
rf_transformer.pca_transformers

defaultdict(<function scripts.util.TreeTransformer.__init__.<locals>.<lambda>()>,
            {12: PCA(n_components=5),
             5: PCA(n_components=5),
             9: PCA(n_components=5),
             11: PCA(n_components=5),
             8: PCA(n_components=5),
             10: PCA(n_components=5),
             2: PCA(n_components=5),
             6: PCA(n_components=5),
             0: PCA(n_components=5),
             4: PCA(n_components=5),
             1: PCA(n_components=5),
             7: PCA(n_components=5),
             3: PCA(n_components=5)})